# Hopfion in a nanodisk

In this tutorial we relax a hopfion in a chiral magnetic nanodisk and calculate its Hopf index. We consider a nanodisk with a radius $r=100 \, \text{nm}$, a thickness $t=70 \, \text{nm}$, and ends at the top and bottom of thickness $d=10 \, \text{nm}$ in which the unit magnetization field $\boldsymbol{m}$ is frozen along the $z$-direction.

This tutorial draws on the works [Y. Liu, R. K. Lake and J. Zang. Binding a hopfion in a chiral magnet nanodisk. Phys. Rev. B 98, 174437 (2019)](https://doi.org/10.1103/PhysRevB.98.174437) and [P. Sutcliffe. Hopfions in chiral magnets. J. Phys. A: Math. Theor. 51 375401 (2018)](https://doi.org/10.1088/1751-8121/aad521).

In [ ]:
import numpy as np
import oommfc as oc
import discretisedfield as df
import micromagneticmodel as mm
from discretisedfield.tools import hopf_index

We define the region 'cylinder', in which the magnetization evoles, as well as 'top' and 'bottom', the two fixed layers.

In [ ]:
r = 100e-9
t = 70e-9
d = 10e-9

region = df.Region(p1=(-r, -r, -d), p2=(r, r, t+d))
subregions = {'bottom': df.Region(p1=(-r, -r, -d), p2=(r, r, 0)),
              'cylinder': df.Region(p1=(-r, -r, 0), p2=(r, r, t)),
              'top': df.Region(p1=(-r, -r, t), p2=(r, r, t+d))}
mesh = df.Mesh(region=region, cell=(2e-9, 2e-9, 2e-9), subregions=subregions)

mesh.k3d.subregions()

We consider a system with symmetric exchange and DMI energy.

In [ ]:
system = mm.System(name='hopfion')

system.energy = (mm.Exchange(A=2.195e-12)
               + mm.DMI(D=3.95e-4, crystalclass='T'))

The magnetization is set to be zero outside of the disk geometry.

In [ ]:
Ms = 3.84e5

def Ms_fun(pos):
    x, y, z = pos
    if (x**2 + y**2)**0.5 < r:
        return Ms
    else:
        return 0

We initialise a hopfion spin texture as a toroidal skyrmion tube with major radius $L=t/2$, skyrmion radius $R=t/4$, and skyrmion domain wall width of $w=t/8$.

In [ ]:
L = t/2
R = t/4
w = t/8

def m_init(pos):
    
    x, y, z = pos

    # Translate to centre of nanodisk
    z -= t/2

    # Angle around origin in x-y-plane
    psi = np.arctan2(y, x)

    # x-coordinate in reference plane with a centre coinciding with the skyrmion's centre
    xPrime = (x - L*np.cos(psi)) * np.cos(psi) + (y - L*np.sin(psi)) * np.sin(psi)

    # Set magnetization in this reference frame
    rho = np.sqrt(xPrime**2 + z**2)
    Phi = np.arctan2(z, xPrime)
    Theta = 2 * np.arctan2(np.sinh(R/w), np.sinh(rho/w))
    myPrime = np.cos(Phi) * np.sin(Theta)
    mz = np.cos(Theta)
    mxPrime = np.sin(Phi) * np.sin(Theta)

    # Transform magnetization back to global coordinates
    mx = mxPrime*np.cos(psi) - myPrime*np.sin(psi)
    my = mxPrime*np.sin(psi) + myPrime*np.cos(psi)
    
    return (mx, my, mz)

m = {'bottom': (0, 0, 1), 'cylinder': m_init, 'top': (0, 0, 1)}
system.m = df.Field(mesh, nvdim=3, value=m, norm=Ms_fun)

We can image the initialised hopfion texture.

In [ ]:
system.m.sel('x').mpl()

We can also calculate the Hopf index of this hopfion texture.

In [ ]:
print(hopf_index(system.m))

We relax the system with the subregions 'top' and 'bottom' fixed.

In [ ]:
md = oc.MinDriver()
md.drive(system, fixed_subregions=['bottom', 'top'])

In [ ]:
system.m.plane('x').mpl()

In [ ]:
print(hopf_index(system.m))